In [1]:
# Import dependencies
import dotenv
import os
import pickle
import py_clob_client
import requests
import prediction_pipeline
from datetime import datetime, timedelta, timezone
from py_clob_client.constants import POLYGON
from py_clob_client.client import ClobClient
from py_clob_client.clob_types import OrderArgs
from py_clob_client.order_builder.constants import BUY
import disk_cache

In [2]:
env = dotenv.dotenv_values(".env")
HOST = "https://clob.polymarket.com"
POLYMARKET_KEY = env["PK"]
CHAIN_ID = POLYGON

# Variables
RISK_TOLERANCE = 0.7  # Represents how much to adjust EV calculations based on the width of the model confidence bounds.

# Create the client
client = ClobClient(HOST, key=POLYMARKET_KEY, chain_id=CHAIN_ID)
client.set_api_creds(client.create_or_derive_api_creds())

# Setup cache
cache = disk_cache.DiskCache(cache_dir="api_cache", expiry_hours=24)

In [3]:
def fetch_all_events_from_gamma():
  """
  Fetch all events from Gamma API.
  """
  events = []
  offset = 0
  limit = 100  # Max results per page
  
  while True:
    url = "https://gamma-api.polymarket.com/events"
    params = {
      "closed": "false",
      "active": "true",
      "limit": limit,
      "offset": offset,
      "order": "createdAt",
      "ascending": "false"
    }
    
    response = requests.get(url, params=params)
    if response.status_code != 200:
      print(f"Error fetching events: {response.status_code}")
      break
        
    batch = response.json()
    if not batch:  # No more results
      break
        
    events.extend(batch)
    offset += limit
      
  return events

def fetch_all_markets(client):
  """
  Fetch all markets from CLOB, iterating through all possible pages.
  """
  markets = []
  next_cursor = ""

  while True:
    response = client.get_markets(next_cursor=next_cursor)
    if not response or 'data' not in response:
      break
    markets.extend(response['data'])

    next_cursor = response.get('next_cursor', '')
    if next_cursor == 'LTE=' or not next_cursor:
      break
  return {market['condition_id']: market for market in markets}

def filter_markets(condition_id_to_market):
    # Filter for active, non-closed markets.
    current_markets = {k: v for k, v in condition_id_to_market.items() if v['active'] and not v['closed']}
    print(f"Found {len(current_markets)} current markets")
    
    # Filter for markets ending in the future. Some markets are kept open past their end date due to a dispute in the resolution.
    future_markets = {k: v for k, v in current_markets.items() if v.get('end_date_iso') and v['end_date_iso'] > datetime.now(timezone.utc).isoformat()}
    print(f"Found {len(future_markets)} future markets")
    
    # Filter for markets ending in the next 1 days.
    near_term_markets = {k: v for k, v in future_markets.items() if v.get('end_date_iso') and v['end_date_iso'] <= (datetime.now(timezone.utc) + timedelta(days=1)).isoformat()}
    print(f"Found {len(near_term_markets)} near term markets")
    
    near_non_sports_markets = {k: v for k, v in near_term_markets.items() if v.get('tags') and 'Sports' not in v['tags']}
    print(f"Found {len(near_non_sports_markets)} near term non-sports markets")
    return near_non_sports_markets

def fetch_all_orderbooks(client, markets):
  # Fetch and attach order books for each market
  print(f"Fetching order books for {len(markets)} markets.")
  token_id_to_book = {}
  for _, market in markets.items():
    tokens = market.get('tokens', [])
    for token in tokens:
      token_id = token.get('token_id', None)
      if token_id:
        try:
          print(f"Fetching order book")
          order_book = client.get_order_book(token_id)
        except py_clob_client.exceptions.PolyApiException as e:
          print(f"Error fetching order book for token {token_id}: {e}")
          continue
        token_id_to_book[token_id] = order_book
  return token_id_to_book

In [4]:
today = datetime.now(timezone.utc).strftime('%Y-%m-%d')
events_pickle_path = f"snapshots/events_snapshot_{today}.pkl"
markets_pickle_path = f"snapshots/markets_snapshot_{today}.pkl"

# Check if events pickle exists
if os.path.exists(events_pickle_path):
  with open(events_pickle_path, 'rb') as f:
    all_events = pickle.load(f)
  print(f"Loaded {len(all_events)} events from {events_pickle_path}")
else:
  # Fetch Events via "Gamma" (using placeholder method)
  all_events = fetch_all_events_from_gamma()
  # Convert list of events to dictionary with event ID as key
  all_events = {event['id']: event for event in all_events}

with open(events_pickle_path, 'wb') as f:
  pickle.dump(all_events, f)
  print(f"Saved {len(all_events)} events to {events_pickle_path}")

# Check if markets pickle exists
if os.path.exists(markets_pickle_path):
  with open(markets_pickle_path, 'rb') as f:
    all_markets = pickle.load(f)
  print(f"Loaded {len(all_markets)} markets from {markets_pickle_path}")
else:
  # Fetch Markets via CLOB
  condition_id_to_market = fetch_all_markets(client)

with open(markets_pickle_path, 'wb') as f:
  pickle.dump(condition_id_to_market, f)
print(f"Saved {len(condition_id_to_market)} markets to {markets_pickle_path}")

condition_id_to_filtered_market = filter_markets(condition_id_to_market)

Saved 1053 events to snapshots/events_snapshot_2025-03-05.pkl
Saved 30434 markets to snapshots/markets_snapshot_2025-03-05.pkl
Found 2920 current markets
Found 2740 future markets
Found 38 near term markets
Found 20 near term non-sports markets


In [5]:
# Create Market Probability Predictions
condition_id_to_prediction = {}
for condition_id, market in condition_id_to_filtered_market.items():
  market_title = market.get('question', '')
  market_description = market.get('description', '')
  prediction_json = prediction_pipeline.create_prediction(market_description, cache=cache)
  if not prediction_json:
    continue
  condition_id_to_prediction[condition_id] = prediction_json

Checking for cached report...
No cached report found. Generating new report...
Writing report...
Report: Based on the search results and the given prediction market details, here is a comprehensive analysis of relevant information:

1. European Central Bank (ECB) Current Policy Rate
Source: Trading Economics, March 2025
Key findings:
- The current ECB deposit facility rate is 2.75% as of January 2025.
- This rate was decreased from 3% in December 2024.
Relevance: Establishes the baseline rate from which any change will be measured.
Quantitative data: Current deposit facility rate = 2.75%
Reliability: High - Trading Economics is a reputable source for economic data.

2. ECB March Meeting Preview
Source: Data Insights Market, March 4, 2025
Key findings:
- The ECB meeting is scheduled for March 5-6, 2025.
- Disinflationary trends and economic uncertainties are noted.
- The ECB's primary objective is to maintain price stability with an inflation target close to but below 2%.
Relevance: Pro

In [6]:
today = datetime.now(timezone.utc).strftime('%Y-%m-%d')
active_order_books_pickle_path = f"snapshots/active_order_books_snapshot_{today}.pkl"

token_id_to_book = {}
# Check if order book pickle exists
if os.path.exists(active_order_books_pickle_path):
  with open(active_order_books_pickle_path, 'rb') as f:
    token_id_to_book = pickle.load(f)
  print(f"Loaded {len(token_id_to_book)} order books from {active_order_books_pickle_path}")
else:
  print("No order book pickle found.")
  # Fetch order books using CLOB client
  token_id_to_book = fetch_all_orderbooks(client, condition_id_to_filtered_market)
  with open(active_order_books_pickle_path, 'wb') as f:
    pickle.dump(token_id_to_book, f)
    print(f"Saved {len(token_id_to_book)} events to {active_order_books_pickle_path}")

No order book pickle found.
Fetching order books for 20 markets.
Fetching order book
Fetching order book
Fetching order book
Fetching order book
Fetching order book
Fetching order book
Fetching order book
Fetching order book
Fetching order book
Fetching order book
Fetching order book
Fetching order book
Fetching order book
Fetching order book
Fetching order book
Fetching order book
Fetching order book
Fetching order book
Fetching order book
Fetching order book
Fetching order book
Fetching order book
Fetching order book
Fetching order book
Fetching order book
Fetching order book
Fetching order book
Fetching order book
Fetching order book
Fetching order book
Fetching order book
Fetching order book
Fetching order book
Fetching order book
Fetching order book
Fetching order book
Fetching order book
Fetching order book
Fetching order book
Fetching order book
Saved 40 events to snapshots/active_order_books_snapshot_2025-03-05.pkl


In [7]:
today = datetime.now(timezone.utc).strftime('%Y-%m-%d')
polymarket_snapshot_pickle_path = f"snapshots/polymarket_snapshot_{today}.pkl"

polymarket_snapshot = {
    "order_books": token_id_to_book,
    "markets": condition_id_to_filtered_market,
    "predictions": condition_id_to_prediction,
}
# Check if market snapshot pickle exists
if os.path.exists(polymarket_snapshot_pickle_path):
    print("already saved snapshot pickle")
else:
  print("No snapshot pickle found.")
  # Fetch order books using CLOB client
  with open(polymarket_snapshot_pickle_path, 'wb') as f:
    pickle.dump(market_snapshot, f)
    print(f"Saved market snapshot pickle to {polymarket_snapshot_pickle_path}")

No snapshot pickle found.


NameError: name 'market_snapshot' is not defined

In [ ]:
possible_positions = []
condition_id_to_market = polymarket_snapshot["markets"]
token_id_to_order_book = polymarket_snapshot["order_books"]
condition_id_to_prediction = polymarket_snapshot["predictions"]

In [ ]:
def calculate_risk_adjusted_ev(prediction, order):
  probability = prediction['probability']
  confidence = prediction['model_confidence']
  uncertainty = prediction['uncertainty']
  uncertainty_range = uncertainty['upper_bound'] - uncertainty['lower_bound']
  edge = probability - float(order.price)
  ev = edge * confidence
  risk_adjusted_ev = ev * (1 - (1 - RISK_TOLERANCE) * uncertainty_range)
  return risk_adjusted_ev

def invert_prediction_probability(pred):
  inverted_pred = {
      "probability": 1 - pred["probability"],
      "model_confidence": pred["model_confidence"],
      "uncertainty": {
          "upper_bound": 1 - pred["uncertainty"]["lower_bound"],
          "lower_bound": 1 - pred["uncertainty"]["upper_bound"],
          "confidence": pred["uncertainty"]["confidence_level"],
      }
  }
  return inverted_pred

In [ ]:
# Create possible position list
ordered_possible_positions = []

for cond_id, market in condition_id_to_market.items():
  prediction = condition_id_to_prediction[cond_id]
  for token in market['tokens']:
    if token['outcome'] == 'Yes':
      yes_token = token
      yes_order_book = token_id_to_order_book[token['token_id']]
      if yes_order_book.asks:
        best_yes_ask = yes_order_book.asks[-1]
        adjusted_ev = calculate_risk_adjusted_ev(prediction, best_yes_ask)
        ordered_possible_positions.append((adjusted_ev, cond_id, yes_token['token_id'], float(best_yes_ask.price), float(best_yes_ask.size)))
    else:
      no_token = token
      no_order_book = token_id_to_order_book[token['token_id']]
      if no_order_book.asks:
        best_no_ask = no_order_book.asks[-1]
        no_pred = invert_prediction_probability(prediction)
        adjusted_ev = calculate_risk_adjusted_ev(no_pred, best_no_ask)
        ordered_possible_positions.append((adjusted_ev, cond_id, no_token['token_id'], float(best_no_ask.price), float(best_no_ask.size)))

ordered_possible_positions.sort(key=lambda x: x[0])

In [ ]:
print(ordered_possible_positions)

In [ ]:
# Create Edge Summaries
edge_summary = []
for prediction_json in condition_id_to_prediction.values():
  yes_prob = float(prediction_json['probability'])
  tokens = market.get('tokens', [])
  for token in tokens:
    outcome = token.get('outcome', '')
    token_id = token.get('token_id', None)
    if token_id:
      try:
        order_book = client.get_order_book(token_id)
        print("Order book:", order_book)
        if order_book.asks:
          best_ask_price = float(order_book.asks[-1].price)
          best_ask_size = float(order_book.asks[-1].size)
        
          # Calculate edge
          if outcome == 'Yes':
            edge = yes_prob - best_ask_price
          else:  # No
            edge = (1 - yes_prob) - best_ask_price
          market_summary = {
            'title': market_title,
            'condition_id': condition_id,
            'probability': prediction_json['probability'],
            'model_confidence': prediction_json['model_confidence'],
            'uncertainty': prediction_json['uncertainty'],
            'token_outcome': outcome,
            'best_ask_price': best_ask_price,
            'best_ask_size': best_ask_size,
            'asks': order_book.asks,
            'bids': order_book.bids,
            'edge': edge
          }
          edge_summary.append(market_summary)
              
      except py_clob_client.exceptions.PolyApiException as e:
         print(f"Error fetching order book for token {token_id}: {e}")
         continue

In [ ]:
print("Edge Summary:", edge_summary)
# Sort edge_summary by largest positive edge
edge_summary_by_edge = sorted(
  [m for m in edge_summary if m['edge'] > 0],
  key=lambda x: x['edge'],
  reverse=True
)
print("Edge Summary sorted by Edge:", edge_summary_by_edge)

# Sort edge_summary by edge times best_ask_size (expected value)
edge_summary_by_ev = sorted(
  [m for m in edge_summary if m['edge'] > 0],
  key=lambda x: x['edge'] * x['best_ask_size'],
  reverse=True
)
print("Edge Summary sorted by EV:", edge_summary_by_ev)